In [2]:
import numpy as np
import pandas as pd
import json
import re

/usr/local/var/pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/var/pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
with open("../data/compound_train.json") as f:
    train = json.load(f)

In [3]:
flatten = lambda l: [item for sublist in l for item in sublist if len(item) is not 0]

In [4]:
Youto_values = []
for item in [entry['Attributes']['用途'] for entry in train['entry']]:
    Youto_values.append(flatten([i.split('。') for i in item]))

In [5]:
Youto_dict = dict(zip([str(entry['WikipediaID']) for entry in train['entry']], Youto_values))

In [4]:
def read_jasonl(filename):
    with open(filename) as f:
        return [json.loads(line.rstrip('\r\n')) for line in f.readlines()]

In [5]:
wiki_data = read_jasonl("../data/jawiki-cirrussearch-dump_of_Compound.jsonl")

## training data 作成

In [8]:
train_df =  pd.DataFrame()
for key, value in Youto_dict.items():
    if len(value) is 0:
        continue
    
    text = [entry for entry in wiki_data if entry['index']['_id'] == key][0]['text']
    clean_text = re.sub(r'\^\s[^\^]+|\\[a-zA-Z0-9]+|{.+}|\"', '', text)
    clean_text = re.sub(r'\s{2,}', ' ', clean_text)
    sentences = list(map(lambda s: s.strip(), re.findall(".*?。", clean_text)))
    
    label = [0] * len(sentences)
    for i, s in enumerate(sentences):
        for train_s in value:
            if train_s in s:
                label[i] = 1
                break
                
    train_df = train_df.append(pd.DataFrame({"_id": key, "sentence": sentences, "label": label}), ignore_index=True)

In [9]:
train_df.groupby('label').count()

,_id,sentence
label,,
0,5157,5157
1,1514,1514


In [11]:
train_df.sort_values('label').to_csv("../data/train_Youto_sentence.csv", index=False)

In [10]:
train_df

,_id,sentence,label
0,1385415,アビエチン酸（Abietic acid）は松脂に含まれる主要な刺激性成分で、ロジンを構成する...,0
1,1385415,構造的には4つのイソプレン単位からなるイソプレノイドでジテルペンの一種である。,0
2,1385415,別名シルビン酸（Sylvic Acid）。,0
3,1385415,アビエチン酸自体は松脂には含まれておらず、ロジン抽出の際にピマール酸（Pimaric aci...,0
4,1385415,アビエチン酸自体は弱い接触性アレルゲンであるが、空気で酸化されると強いアレルゲンになる。,0
5,1385415,アルコール、アセトン、エーテルなどに溶解する。,0
6,1385415,商業的には、ガラス質、もしくは一部結晶化された状態で売られている。,0
7,1385415,脂入りはんだのフラックスの主要成分である[1]ほか、ラッカー、ワニス、石鹸や樹脂酸金属塩の製...,1
8,1023103,リナロール (linalool) は分子式 C10H18O で表されるモノテルペンアルコール...,0
9,1023103,スズラン、ラベンダー、ベルガモット様の芳香をもつため、大量に香料として利用されている。,1


## 全ての記事データをSentenceごとにsplit

In [9]:
wiki_df = pd.DataFrame()
for article in wiki_data:
    clean_text = re.sub(r'\^\s[^\^]+', '', article['text'])
    clean_text = re.sub(r'\\[a-zA-Z0-9]+', '', clean_text)
    clean_text = re.sub(r'{.+}', '', clean_text)
    clean_text = re.sub(r'\"', '', clean_text)
    clean_text = re.sub(r'\s{2,}', ' ', clean_text)
    clean_text = clean_text.replace('（', '(').replace('）', ')')
    sentences = list(map(lambda s: s.strip(), re.findall(".*?。", clean_text)))
    
    wiki_df = wiki_df.append(pd.DataFrame({"_id": article['index']['_id'], "sentence": sentences}))

In [10]:
wiki_df.to_csv("../data/wikitext_split_sentence.csv", index=False)